In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score
import cv2
import re
# import os
import glob
# import re
# import numpy as np
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
import tifffile as tiff
# import os
# import numpy as np
# import glob
# from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
# import rasterio

def load_image(filepath):
    # Load the image using PIL
    image = np.array(Image.open(filepath).convert('L'))
    
    # Apply Otsu's thresholding
    t, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Convert to binary (0 and 1)
    binary_image = (binary_image > 0).astype(np.uint8)
    return binary_image

def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred))

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)

# Paths to the folders containing the predictions and ground truths


In [2]:
# gt_folder = './GEE_Masks/GEE_resized/'
# pred_folder = './GEE_Output/Adversarial'

# # List all files in the prediction folder
# # pred_files = [f for f in os.listdir(pred_folder) if f.endswith('.tif')]

# # Initialize metrics



In [3]:
# # Lists to store metrics
# dice_scores = []
# iou_scores = []
# pixel_accuracies = []
# precisions = []
# recalls = []
# f1_scores = []
# specificities = []

# for i in range(0, 1263):  # Adjust the range based on the number of files
#     gt_filename = f'NDWI_Mask_{i}_resized.tif'
#     pred_filename = f'dense_Predicted_Image_{i}.tif'
    
#     gt_path = os.path.join(gt_folder, gt_filename)
#     pred_path = os.path.join(pred_folder, pred_filename)
    
#     # Load the prediction and ground truth images
#     pred_image = load_image(pred_path)
#     gt_image = load_image(gt_path)
    
#     # Flatten the images
#     pred_flat = pred_image.flatten()
#     gt_flat = gt_image.flatten()
    
#     # Calculate metrics
#     dice = dice_coefficient(gt_flat, pred_flat)
#     iou = jaccard_score(gt_flat, pred_flat, average='macro')
#     accuracy = pixel_accuracy(gt_flat, pred_flat)
#     precision = precision_score(gt_flat, pred_flat, average='macro', zero_division=0)
#     recall = recall_score(gt_flat, pred_flat, average='macro')
#     f1 = f1_score(gt_flat, pred_flat, average='macro')
#     spec = specificity(gt_flat, pred_flat)
    
#     # Store metrics
#     dice_scores.append(dice)
#     iou_scores.append(iou)
#     pixel_accuracies.append(accuracy)
#     precisions.append(precision)
#     recalls.append(recall)
#     f1_scores.append(f1)
#     specificities.append(spec)
    
#     # Print metrics for the current image
#     print(f"Metrics for {gt_filename} and {pred_filename}:")
#     print(f"  Dice Coefficient: {dice:.4f}")
#     print(f"  IoU: {iou:.4f}")
#     print(f"  Pixel Accuracy: {accuracy:.4f}")
#     print(f"  Precision: {precision:.4f}")
#     print(f"  Recall: {recall:.4f}")
#     print(f"  F1 Score: {f1:.4f}")
#     print(f"  Specificity: {spec:.4f}")
#     print()

# # Calculate and print average metrics
# average_dice = np.mean(dice_scores)
# average_iou = np.mean(iou_scores)
# average_accuracy = np.mean(pixel_accuracies)
# average_precision = np.mean(precisions)
# average_recall = np.mean(recalls)
# average_f1 = np.mean(f1_scores)
# average_specificity = np.mean(specificities)

# print("Average metrics:")
# print(f"  Average Dice Coefficient: {average_dice:.4f}")
# print(f"  Average IoU: {average_iou:.4f}")
# print(f"  Average Pixel Accuracy: {average_accuracy:.4f}")
# print(f"  Average Precision: {average_precision:.4f}")
# print(f"  Average Recall: {average_recall:.4f}")
# print(f"  Average F1 Score: {average_f1:.4f}")
# print(f"  Average Specificity: {average_specificity:.4f}")

# # Model's overall accuracy
# print(f"Model's Overall Accuracy: {average_accuracy:.4f}")

In [4]:
# # Print average metrics
# print("Average Metrics:")
# print(f"  Average Dice Coefficient: {np.mean(dice_scores):.4f}")
# print(f"  Average IoU: {np.mean(iou_scores):.4f}")
# print(f"  Average Pixel Accuracy: {np.mean(pixel_accuracies):.4f}")
# print(f"  Average Precision: {np.mean(precisions):.4f}")
# print(f"  Average Recall: {np.mean(recalls):.4f}")
# print(f"  Average F1 Score: {np.mean(f1_scores):.4f}")
# print(f"  Average Specificity: {np.mean(specificities):.4f}")

In [2]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def pixel_accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def specificity(y_true, y_pred):
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tn / (tn + fp)


# Directories for ground truth and prediction masks
gt_folder = './GEE_Masks/GEE_resized/'
pred_folder = './GEE_Output/Clean'

# Initialize metric lists
dice_scores = []
iou_scores = []
pixel_accuracies = []
precisions = []
recalls = []
f1_scores = []
specificities = []

# Get all prediction files
pred_files = glob.glob(os.path.join(pred_folder, 'dense_*.tif'))

for pred_file in pred_files:
    # Extract the number 'i' from the prediction filename
    match = re.search(r'dense_(\d+).tif', os.path.basename(pred_file))
    if match:
        i = match.group(1)
        gt_filename = f'NDWI_Mask_{i}_resized.tif'
        gt_path = os.path.join(gt_folder, gt_filename)
        
        # Check if the ground truth file exists
        if os.path.exists(gt_path):
            # Load the prediction and ground truth images
            pred_image = load_image(pred_file)
            gt_image = load_image(gt_path)
            print(np.max(pred_image))
            print(np.min(pred_image))
            # Flatten the images
            pred_flat = pred_image.flatten()
            gt_flat = gt_image.flatten()
            
            # Calculate metrics
            dice = dice_coefficient(gt_flat, pred_flat)
            iou = jaccard_score(gt_flat, pred_flat, average='macro')
            accuracy = pixel_accuracy(gt_flat, pred_flat)
            precision = precision_score(gt_flat, pred_flat, average='macro', zero_division=0)
            recall = recall_score(gt_flat, pred_flat, average='macro')
            f1 = f1_score(gt_flat, pred_flat, average='macro')
            spec = specificity(gt_flat, pred_flat)
            
            # Store metrics
            dice_scores.append(dice)
            iou_scores.append(iou)
            pixel_accuracies.append(accuracy)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1)
            specificities.append(spec)
            
            # Print metrics for the current image
            print(f"Metrics for {gt_filename} and {pred_file}:")
            print(f"  Dice Coefficient: {dice:.4f}")
            print(f"  IoU: {iou:.4f}")
            print(f"  Pixel Accuracy: {accuracy:.4f}")
            print(f"  Precision: {precision:.4f}")
            print(f"  Recall: {recall:.4f}")
            print(f"  F1 Score: {f1:.4f}")
            print(f"  Specificity: {spec:.4f}")
            print()
        else:
            print(f"Ground truth file {gt_path} not found for prediction file {pred_file}.")

# Calculate and print average metrics
average_dice = np.mean(dice_scores)
average_iou = np.mean(iou_scores)
average_accuracy = np.mean(pixel_accuracies)
average_precision = np.mean(precisions)
average_recall = np.mean(recalls)
average_f1 = np.mean(f1_scores)
average_specificity = np.mean(specificities)

print("Average Metrics:")
print(f"  Dice Coefficient: {average_dice:.4f}")
print(f"  IoU: {average_iou:.4f}")
print(f"  Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Precision: {average_precision:.4f}")
print(f"  Recall: {average_recall:.4f}")
print(f"  F1 Score: {average_f1:.4f}")
print(f"  Specificity: {average_specificity:.4f}")

1
0
Metrics for NDWI_Mask_10_resized.tif and ./GEE_Output/Clean\dense_10.tif:
  Dice Coefficient: 0.6948
  IoU: 0.6454
  Pixel Accuracy: 0.8106
  Precision: 0.8616
  Recall: 0.7646
  F1 Score: 0.7787
  Specificity: 0.9831

1
0
Metrics for NDWI_Mask_1008_resized.tif and ./GEE_Output/Clean\dense_1008.tif:
  Dice Coefficient: 0.8626
  IoU: 0.8685
  Pixel Accuracy: 0.9799
  Precision: 0.9730
  Recall: 0.8887
  F1 Score: 0.9259
  Specificity: 0.9975

1
0
Metrics for NDWI_Mask_1010_resized.tif and ./GEE_Output/Clean\dense_1010.tif:
  Dice Coefficient: 0.9900
  IoU: 0.9851
  Pixel Accuracy: 0.9933
  Precision: 0.9948
  Recall: 0.9902
  F1 Score: 0.9925
  Specificity: 0.9998

1
0
Metrics for NDWI_Mask_1012_resized.tif and ./GEE_Output/Clean\dense_1012.tif:
  Dice Coefficient: 0.8542
  IoU: 0.8632
  Pixel Accuracy: 0.9819
  Precision: 0.9286
  Recall: 0.9162
  F1 Score: 0.9223
  Specificity: 0.9914

1
0
Metrics for NDWI_Mask_1020_resized.tif and ./GEE_Output/Clean\dense_1020.tif:
  Dice Coeffic

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_807_resized.tif and ./GEE_Output/Clean\dense_807.tif:
  Dice Coefficient: 0.9160
  IoU: 0.9135
  Pixel Accuracy: 0.9836
  Precision: 0.9653
  Recall: 0.9424
  F1 Score: 0.9535
  Specificity: 0.9939

1
0
Metrics for NDWI_Mask_808_resized.tif and ./GEE_Output/Clean\dense_808.tif:
  Dice Coefficient: 0.8972
  IoU: 0.8873
  Pixel Accuracy: 0.9667
  Precision: 0.9558
  Recall: 0.9234
  F1 Score: 0.9387
  Specificity: 0.9888

1
0
Metrics for NDWI_Mask_810_resized.tif and ./GEE_Output/Clean\dense_810.tif:
  Dice Coefficient: 0.0001
  IoU: 0.4832
  Pixel Accuracy: 0.9665
  Precision: 0.5000
  Recall: 0.4832
  F1 Score: 0.4915
  Specificity: 0.9665

1
0


c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_814_resized.tif and ./GEE_Output/Clean\dense_814.tif:
  Dice Coefficient: 0.7979
  IoU: 0.7766
  Pixel Accuracy: 0.9093
  Precision: 0.8398
  Recall: 0.9184
  F1 Score: 0.8697
  Specificity: 0.9036

1
0
Metrics for NDWI_Mask_837_resized.tif and ./GEE_Output/Clean\dense_837.tif:
  Dice Coefficient: 0.7394
  IoU: 0.7231
  Pixel Accuracy: 0.8829
  Precision: 0.7987
  Recall: 0.8993
  F1 Score: 0.8320
  Specificity: 0.8737

1
0
Metrics for NDWI_Mask_838_resized.tif and ./GEE_Output/Clean\dense_838.tif:
  Dice Coefficient: 0.3728
  IoU: 0.5481
  Pixel Accuracy: 0.8721
  Precision: 0.6178
  Recall: 0.7840
  F1 Score: 0.6508
  Specificity: 0.8831

1
0
Metrics for NDWI_Mask_846_resized.tif and ./GEE_Output/Clean\dense_846.tif:
  Dice Coefficient: 0.5699
  IoU: 0.6779
  Pixel Accuracy: 0.9586
  Precision: 0.9512
  Recall: 0.7032
  F1 Score: 0.7740
  Specificity: 0.9982

1
0
Metrics for NDWI_Mask_847_resized.tif and ./GEE_Output/Clean\dense_847.tif:
  Dice Coefficient: 0.54

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_88_resized.tif and ./GEE_Output/Clean\dense_88.tif:
  Dice Coefficient: 0.1000
  IoU: 0.5000
  Pixel Accuracy: 1.0000
  Precision: 0.5000
  Recall: 0.5000
  F1 Score: 0.5000
  Specificity: 1.0000

1
0
Metrics for NDWI_Mask_883_resized.tif and ./GEE_Output/Clean\dense_883.tif:
  Dice Coefficient: 0.0092
  IoU: 0.3130
  Pixel Accuracy: 0.6220
  Precision: 0.5019
  Recall: 0.7050
  F1 Score: 0.3879
  Specificity: 0.6217

1
0
Metrics for NDWI_Mask_890_resized.tif and ./GEE_Output/Clean\dense_890.tif:
  Dice Coefficient: 0.0001
  IoU: 0.4658
  Pixel Accuracy: 0.9316
  Precision: 0.4997
  Recall: 0.4660
  F1 Score: 0.4823
  Specificity: 0.9320

1
0
Metrics for NDWI_Mask_893_resized.tif and ./GEE_Output/Clean\dense_893.tif:
  Dice Coefficient: 0.7680
  IoU: 0.7692
  Pixel Accuracy: 0.9256
  Precision: 0.9227
  Recall: 0.8232
  F1 Score: 0.8618
  Specificity: 0.9866

1
0
Metrics for NDWI_Mask_9_resized.tif and ./GEE_Output/Clean\dense_9.tif:
  Dice Coefficient: 0.9280
  I

c:\Users\ADMIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for NDWI_Mask_95_resized.tif and ./GEE_Output/Clean\dense_95.tif:
  Dice Coefficient: 0.5000
  IoU: 0.5000
  Pixel Accuracy: 1.0000
  Precision: 0.5000
  Recall: 0.5000
  F1 Score: 0.5000
  Specificity: 1.0000

0
0
Metrics for NDWI_Mask_950_resized.tif and ./GEE_Output/Clean\dense_950.tif:
  Dice Coefficient: 1.0000
  IoU: 1.0000
  Pixel Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1 Score: 1.0000
  Specificity: 1.0000

0
0
Metrics for NDWI_Mask_955_resized.tif and ./GEE_Output/Clean\dense_955.tif:
  Dice Coefficient: 1.0000
  IoU: 1.0000
  Pixel Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1 Score: 1.0000
  Specificity: 1.0000

0
0
Metrics for NDWI_Mask_957_resized.tif and ./GEE_Output/Clean\dense_957.tif:
  Dice Coefficient: 0.0064
  IoU: 0.4997
  Pixel Accuracy: 0.9994
  Precision: 0.4997
  Recall: 0.5000
  F1 Score: 0.4999
  Specificity: 1.0000

0
0
Metrics for NDWI_Mask_960_resized.tif and ./GEE_Output/Clean\dense_960.tif:
  Dice Coefficient: 0.3333

In [3]:
print("Average metrics:")
print(f"  Average Dice Coefficient: {average_dice:.4f}")
print(f"  Average IoU: {average_iou:.4f}")
print(f"  Average Pixel Accuracy: {average_accuracy:.4f}")
print(f"  Average Precision: {average_precision:.4f}")
print(f"  Average Recall: {average_recall:.4f}")
print(f"  Average F1 Score: {average_f1:.4f}")
print(f"  Average Specificity: {average_specificity:.4f}")

# Model's overall accuracy
print(f"Model's Overall Accuracy: {average_accuracy:.4f}")

Average metrics:
  Average Dice Coefficient: 0.7696
  Average IoU: 0.8230
  Average Pixel Accuracy: 0.9554
  Average Precision: 0.8959
  Average Recall: 0.8688
  Average F1 Score: 0.8652
  Average Specificity: 0.9708
Model's Overall Accuracy: 0.9554
